In [10]:
import pandas as pd
import requests 
from typing import Tuple, Dict, List 
import json  
from IPython.display import display, JSON  
import os     
from dotenv import load_dotenv   
import openai

In [12]:
%%writefile .env
OPENAI_API_KEY="lkolkdflkl"
OPENWEATHER_API_KEY="kskadmlkmldk"

Overwriting .env


In [13]:
class Safety:
    def __init__(self):
        self.weather_api_key = OPENWEATHER_API_KEY
        self.travel_api = "https://www.travel-advisory.info/api" 

    def check_country(self, country: str) -> Tuple[bool , dict]:  
        
        try:
           
            advisory_response = request.get(f"{self.travel_api}?country={country}")
            advisory_data = advisory_response.json()  
            advisory_scroe = advisory_data['data'][country]['advisory']['score']

            
            capital = self._get_capital(country)
            weather_url = f"https://api.openweathermap.org/data/2.5/weather?q={capital}&appid={self.OPENWEATHER_API_KEY}" 
            weather_response = requests.get(weather_url)
            weather_data = weather_response.json() 
            weather_alerts = weather_data.get('alerts' , []) 

            is_safe = (advisory_scroe <= 2.5 ) and (not weather_alerts) 

            return safety_report, {
                "advisory_score": advisory_scroe,
                "weather_alerts": weather_alerts,
                "capital_checked": capital
            }
        except Exception as e :
            print(f"API Error : {str(e)}")
            return True ,{"error" : str(e)}

    def _get_capital(self , country: str) -> str :
           
        capitals = {
            "South Korea": "Seoul",
            "Japan": "Tokyo",
            "Indonesia": "Jakarta",
            "Morocco": "Rabat",
            "Egypt": "Cairo",
            "South Africa": "Pretoria",
            "Turkey": "Ankara",
            "Greece": "Athens",
            "France": "Paris",
            "Spain": "Madrid",
            "Germany": "Berlin",
            "United States": "Washington",
            "Brazil": "Brasília",
            "Mexico": "Mexico City"
        }
        return capitals.get(country, country.split()[0])

In [14]:
class Zawal:
    def __init__(self):
        self.SafteyChecker = Safety() 
        openai.api_key = OPENAI_API_KEY
        self.df = pd.read_csv("zawal - FinalData") 

    def getDefaultPrefs(self) -> Dict:
        """Returns default travel preferences"""
        return {
            'travel_type': 'solo',
            'has_kids': False,
            'budget': 500,
            'season': 'any',
            'age': 30
        }

    def GetSuggestions(self, UserInput: Dict) -> Dict:
        country = UserInput.get('Country')
        userPrefs = UserInput.get('preferences', self.getDefaultPrefs())
        is_safe, safety_report = self.SafteyChecker.check_country(UserInput['Country'])

        if is_safe:
            return {
                # -> what will be returned from Recommender System
            }
        else:
            alternatives = self.SuggestAlternatives(userPrefs, saftey_report)            
            return {
                "Status": "Unsafe",
                "Message": "Saftey concerns with your selected country",
                "Saftey_report": saftey_report,
                "Alternatives": alternatives[:3]
            }

    def GetCountryDetails(self, country: str, userPrefs: Dict) -> Dict:
        try :
            countryData = self.df[self.df['Country'] == country]
            
            if countryData.empty:
                return {"error": "Country not found in database" , "code" :404 }
    
            scores = self.calculateCountryScores(userPrefs)
            CountryScore = scores[scores['Country'] == country].iloc[0]
    
            ai_analysis = self.generateCountryDetails(country, userPrefs)
    
            return {
                'adjustedScore': round(CountryScore['Adjusted Score'], 2),
                'bestScore': round(CountryScore['Best Score'], 2),
                'preferenceMatch': round(CountryScore['Preference Score'], 2),
                'avgAge': round(CountryScore['Avg Age'], 2),
                'budgetRange': f"{round(countryData['Low Budget'].mean(), 0)}-{round(countryData['High Budget'].mean(), 0)}",
                'bestSeasons': f"{countryData['First Season'].mode()[0]}/{countryData['Second Season'].mode()[0]}",
                'kid_friendly': f"{round((countryData['Kids Friendly'] == 'Yes').mean() * 100, 0)}%",
                'aiAnalysis': ai_analysis
            }
        except Exception as e:
            return {"error": str(e), "code": 500}             
        
    def SuggestAlternatives(self, userPrefs: Dict, saftey_report: Dict) -> List[Dict]:
        topCountries = self.getTopCountriesByScore(userPrefs, n=5)
        safeAlternatives = []
        for countryData in topCountries:
            country = countryData['country']
            is_safe, _ = self.SafteyChecker.check_country(country)
            if is_safe:
                safeAlternatives.append(countryData)
                if len(safeAlternatives) >= 3:
                    break
                    
        return safeAlternatives                
        
    def getTopCountriesByScore(self, userPrefs: Dict = None, n=3) -> List[Dict]:
        if userPrefs is None:
            userPrefs = self.getDefaultPrefs()

        countryScores = self.calculateCountryScores(userPrefs)
        topCountries = countryScores.sort_values(by='Adjusted Score', ascending=False).head(n)
        return self.addAiInsights(topCountries, userPrefs)

    def calculateCountryScores(self, userPrefs: Dict) -> pd.DataFrame:
        agfuncs = {
            'Solo traveller score': 'mean',
            'Number of Visitors': 'mean',
            'Activity Duration (in Hrs)': 'mean',
            'Kids Friendly': lambda x: (x == 'Yes').mean(),
            'Average age': 'mean',
            'Low Budget': 'mean',
            'High Budget': 'mean',
            'First Season': lambda x: x.mode()[0],
            'Second Season': lambda x: x.mode()[0]
        }

        countryStats = self.df.groupby('Country').agg(agfuncs).reset_index()
        countryStats['Preference Score'] = 1.0

        if userPrefs['travel_type'] == 'family':
            countryStats['Preference Score'] *= (
                0.7 * countryStats['Kids Friendly'] + 
                0.3 * (1 - countryStats['Solo traveller score'])
            )
        else:
            countryStats['Preference Score'] *= countryStats['Solo traveller score']

        userAge = userPrefs.get('age', 30)
        ageDiff = abs(userAge - countryStats['Average age'])
        countryStats['Preference Score'] *= 1.0 - (ageDiff/100)

        userBudget = userPrefs.get('budget', 500)
        countryStats['Budget Score'] = countryStats.apply(
            lambda row: self._calculate_budget_score(
                userBudget,
                row['Low Budget'],
                row['High Budget']
            ),
            axis=1
        )
        countryStats['Preference Score'] *= countryStats['Budget Score']

        season = userPrefs.get('season', 'any')
        if season != 'any':
            seasonMatch = countryStats.apply(
                lambda x: 1.2 if season in [x['First Season'], x['Second Season']] else 0.8,
                axis=1   
            )
            countryStats['Preference Score'] *= seasonMatch

        baseWeights = {
            'Solo traveller score': 0.3,
            'Number of Visitors': 0.2,
            'Activity Duration (in Hrs)': 0.2,
            'Kids Friendly': 0.1,
            'Average age': 0.1,
            'Low Budget': 0.05,
            'High Budget': 0.05
        }

        countryStats['Base Score'] = sum(
            countryStats[col] * weight for col, weight in baseWeights.items()
        )

        countryStats['Adjusted Score'] = (
            0.6 * countryStats['Base Score'] + 
            0.4 * countryStats['Preference Score']
        )
        
        return countryStats

    def addAiInsights(self, topCountries: pd.DataFrame, userPrefs: Dict) -> List[Dict]:
        prompt = f"""
        Analyze these countries for a traveler with preferences:
        - Type: {'Family' if userPrefs.get('has_kids', False) else userPrefs.get('travel_type', 'solo')}
        - Budget: {userPrefs.get('budget', 500)}
        - Season: {userPrefs.get('season', 'any')}
        - Age: {userPrefs.get('age', 30)}
        """

        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            response_format={"type": "json_object"}
        )        

        insights = json.loads(response.choices[0].message.content)
        
        results = []
        for _, row in topCountries.iterrows():
            results.append({
                'country': row['Country'],
                'adjusted_score': round(row['Adjusted Score'], 2),
                'base_score': round(row['Base Score'], 2),
                'preference_match': round(row['Preference Score'], 2),
                'average_age': round(row['Average age'], 1),
                'budget_range': f"{round(row['Low Budget'], 0)}-{round(row['High Budget'], 0)}",
                'best_seasons': f"{row['First Season']}/{row['Second Season']}",
                'kid_friendly': f"{round(row['Kids Friendly']*100, 0)}%",
                'ai_analysis': insights.get(row['Country'], {})
            })
        
        return results

    def _calculate_budget_score(self, user_budget: float, low_budget: float, high_budget: float) -> float:
        
        budget_range = high_budget - low_budget
        if budget_range <= 0:  
            return 0.5
        
        if user_budget >= low_budget and user_budget <= high_budget:
            return 1.0  
        elif user_budget < low_budget:
            
            return max(0, 1 - (low_budget - user_budget) / (low_budget + 1))
        else:
            
            return max(0, 1 - (user_budget - high_budget) / (high_budget + 1))